In [4]:
import os
import pandas as pd
import plotly.graph_objects as go


In [5]:
# specify the directory containing the CSV files
directory = '../../data/quarterly_financials'

# create an empty list to store the dataframes
dfs = []

# loop over the CSV files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # read the CSV file into a dataframe and append it to the list
        path = os.path.join(directory, filename)
        df = pd.read_csv(path)
        dfs.append(df)

# concatenate the dataframes into a single dataframe
quarterly_financials = pd.concat(dfs, ignore_index=True)
# sort
quarterly_financials = quarterly_financials.sort_values(
    by=['REPDTE', 'CERT'], ascending=[False, True])
quarterly_financials = quarterly_financials.reset_index(drop=True)

In [6]:
# print the combined dataframe
display(quarterly_financials.shape)
display(quarterly_financials.head())

(1000105, 16)

,ZIP,BKCLASS,EQR,REPDTE,DEPDOM,ASSET,STNAME,EQ,NAME,CITY,ADDRESS,ENDEFYMD,CERT,ESTYMD,LIAB,ID
0,2111,SM,8.918529,20221231,163284000.0,298020000,MASSACHUSETTS,26579000.0,STATE STREET BANK&TRUST CO,BOSTON,1 LINCOLN ST,99991231.0,14,17920101,271441000.0,14_20221231
1,36830,SM,6.446081,20221231,952037.0,1023366,ALABAMA,65967.0,AUBURNBANK,AUBURN,100 N GAY ST,99991231.0,35,19070103,957399.0,35_20221231
2,36732,NM,7.323604,20221231,407949.0,444822,ALABAMA,32577.0,ROBERTSON BANKING CO,DEMOPOLIS,216 N WALNUT AVE,99991231.0,39,18700101,412245.0,39_20221231
3,36867,NM,-3.191064,20221231,266874.0,265272,ALABAMA,-8465.0,PHENIX-GIRARD BANK,PHENIX CITY,801 13TH ST,99991231.0,41,19040504,273737.0,41_20221231
4,36401,NM,7.197104,20221231,70649.0,76239,ALABAMA,5487.0,BANK OF EVERGREEN,EVERGREEN,146 W FRONT ST,99991231.0,49,19320901,70752.0,49_20221231


In [7]:
display(len(quarterly_financials['BKCLASS'].unique()))
display(quarterly_financials['BKCLASS'].unique())

7

array(['SM', 'NM', 'N', 'SI', 'SB', 'SL', 'OI'], dtype=object)

In [18]:
# select date
filtered_df = quarterly_financials[quarterly_financials['REPDTE'] == 20221231]

# aggregte on equity, liabilities and assets and rename
mean_values = filtered_df.groupby('STNAME')[['EQ', 'LIAB', 'ASSET']].mean().round(0).reset_index()
mean_values.rename(columns={'EQ': 'Equity', 'LIAB': 'Liabilities', 'ASSET': 'Assets'}, inplace=True)

# format thousand sep
mean_values['Equity'] = mean_values['Equity'].apply(lambda x: '{:,}'.format(int(x)))
mean_values['Liabilities'] = mean_values['Liabilities'].apply(lambda x: '{:,}'.format(int(x)))
mean_values['Assets'] = mean_values['Assets'].apply(lambda x: '{:,}'.format(int(x)))

# alternating row color
row_colors = ['lavender', 'white'] * int(len(mean_values)/2 + 1)
row_colors = row_colors[:len(mean_values)]

# create the tables
table_equity = go.Table(header=dict(values=['State Name', 'Equity'], fill_color='#118C4F', align='left', font=dict(color='white')),
                        cells=dict(values=[mean_values['STNAME'], mean_values['Equity']], fill_color=[row_colors], align='left'), visible=True)

table_liabilities = go.Table(header=dict(values=['State Name', 'Liabilities'], fill_color='#118C4F', align='left', font=dict(color='white')),
                             cells=dict(values=[mean_values['STNAME'], mean_values['Liabilities']], fill_color=[row_colors], align='left'), visible=False)

table_assets = go.Table(header=dict(values=['State Name', 'Assets'], fill_color='#118C4F', align='left', font=dict(color='white')),
                        cells=dict(values=[mean_values['STNAME'], mean_values['Assets']], fill_color=[row_colors], align='left'), visible=False)

# add to fig
fig = go.Figure(data=[table_equity, table_liabilities, table_assets])

# dropdown
dropdown_buttons = [
    dict(label="Equity", method="update", args=[{"visible": [True, False, False]}, {"title": "Average Equity"}]),
    dict(label="Liabilities", method="update", args=[{"visible": [False, True, False]}, {"title": "Average Liabilities"}]),
    dict(label="Assets", method="update", args=[{"visible": [False, False, True]}, {"title": "Average Assets"}]),
]

fig.update_layout(updatemenus=[dict(buttons=dropdown_buttons, showactive=True, x=0.8, y=1.1)])
fig.add_annotation(text="Figures in millions", xref="paper", yref="paper", x=0, y=-0.15, showarrow=False)

# plot
fig.update_layout(title="Average Equity") # initial
fig.show()